## LT
#### 已经生成
1. random
2. mif
#### target 1
1. 完成简单的平台流程， 
这里一个m9k只能使用5*188字节的数据，主要是最长只能有1024*8随意有点麻烦
使用k=16，
需要存16*188*8的原始数据包，这里为了简化，因为这里都大部分可以使用流水线的结构。    
这里主要假设，输入out的数据流的速度小于内部存储的数据流。   


In [ ]:
## 首先定义一下主要用的变量
k=16
## 生成的数据n是无穷
n= float('inf')


In [ ]:
##这里使用verilog 生成 存储k*5*188*8字节的数据，存储到了k*188，或者finished信号出现的时候停止。
from write_verilog import *
from math import *
class dma_source:
    def __init__(self,k,module_name=None,):
        ## 这个是常用的数据
        self.k=k
        if module_name==None:
            self.module_name=f"source_dma_{k}"
        self.ram_width=8
        self.ram_deepth=1024
        self.ram_add_wid=ceil(log2(self.ram_deepth))
        
    def generate_verilog(self,file_name="dma_source.v"):
        k=self.k
        ram_number=self.k
        ram_number_reg=ceil(log2(self.k))
        #例化k个ram，然后使用逻辑导入正确的，直到读入全部的k
        para_list=[]
        para_list+=[("input","wire","","clk")]
        para_list+=[("input","wire","","rst")]
        para_list+=[("input","wire","","data_load_en")]
        para_list+=[("input","wire",f"[{self.ram_width-1}:0]","data")]###如果数据有效就把这个写入
        
        para_list+=[("input","wire","","control_en")]### 控制这个模块的有效
        ## ram 接口
        para_list+=[("output","","","ram_wr_en")]
        
        para_list+=[("output","reg",f"[{self.ram_add_wid-1}:0]","ram_address")]
        para_list+=[("output","reg",f"[{ram_number_reg-1}:0]","ram_selcet")]
        para_list+=[("output","",f"[{self.ram_width-1}:0]","data_ram")]
        
        ## 相当于地址译码器，
        
        
        comment_list=[]
        comment_list+=[f"the number of ram is {k}"]
        comment_list+=[f"this is the ram wr interface with the ram  a little bit like dma"]

        with open(file_name,'w') as fd:
            gv=verilog(fd)
            gv.write_v.write_comment(comment_list)
            #gv.if_begin("posedge clk")
            with gv.module_announcement(self.module_name,para_list):
                
                
                
                with gv.always_begin("posedge clk"):
                    
                    
                        
                    
                    with gv.if_begin("rst"):
                        
                    
                    
                        connect_list=[]
                        connect_list+=[("ram_selcet",f"{ram_number_reg}'d0")]
                        connect_list+=[("ram_address",f"{self.ram_add_wid}'d0")]
                        gv.write_v.write_noblock(connect_list)
                    with gv.if_begin("ram_wr_en"):
                        with gv.if_begin(f"ram_address<={self.ram_add_wid}'d{5*188-1}"):
                            connect_list=[]
                            connect_list+=[(f"ram_selcet","ram_selcet+1'b1")]
                            connect_list+=[("ram_address",f"{self.ram_add_wid}'d0")]
                            gv.write_v.write_noblock(connect_list)
                            
                        with gv.else_begin():
                            connect_list=[]
                            
                            connect_list+=[("ram_address","ram_address+1'b1")]
                            gv.write_v.write_noblock(connect_list)


                connect_list=[]
                connect_list+=[("ram_wr_en","data_load_en&&control_en")]
                connect_list+=[("ram_data","data")]
                gv.write_v.write_wire_connect(connect_list)
                


In [ ]:
from write_verilog import *
from math import *
class ram_k_source:
    def __init__(self,k,module_name=None):
        '''
        初始化，8*1024 的ram模块k个，这个主要麻烦的地方在于线的赋值，然后重复的模块的线的赋值
        接口是一个ram_selct,ram_address,wr_en,rd_en,假设这输出是有一个clock的延迟相应的，这个是为了高平率的扩展
        '''
        self.k=k
        if module_name==None:
            self.module_name=f"ram_{k}_source_2port"
        self.file_name=self.module_name+'.v'
        self.ram_width=8
        self.ram_deepth=1024
        self.ram_add_wid=ceil(log2(self.ram_deepth))
    def generate_verilog(self):
        '''
        ram_inteface:
        @parameter  address input [9:0]
        @parameter  clock sys_clk
        @parameter  data data_in input [7:0]

        @parameter  rden  output
        @parameter  wren input
        @parameter  q   output data [7:0]
        '''
        k=self.k
        ram_number_reg=ceil(log2(self.k))
        file_name=self.file_name
        with open(file_name,"w") as fd:
            #定义输入输出端口
            gv=verilog(fd)
            comment_list=[]
            comment_list+=[f"inst the {k} ram"]
            comment_list+=[f"k ram share the same data,q,rw with special control with ram_select_wr,ram_select_rd,"]
            gv.write_v.write_comment(comment_list)
            para_list=[]
            para_list+=[("input","wire","","clk")]
            para_list+=[("input","wire","","wr_en")]
            para_list+=[("input","wire",f"[{self.ram_add_wid-1}:0]","address_wr")]
            para_list+=[("input","wire",f"[{self.ram_width-1}:0]","data")]
            para_list+=[("input","wire","","rd_en")]
            para_list+=[("output","wire",f"[{self.ram_width-1}:0]","q")]
            para_list+=[("input","wire",f"[{self.ram_add_wid-1}:0]","address_rd")]

            para_list+=[("input","wire",f"[{ram_number_reg-1}:0]","ram_select_wr")]
            para_list+=[("input","wire",f"[{ram_number_reg-1}:0]","ram_select_rd")]

            

            with gv.module_announcement(self.module_name,para_list):
                #例化k个ram，并且给每个ram都写上
                module_inst_name="ram_source_2port"
                interface_list=["rdaddress","wraddress","clock","data","rden","wren","wren"]
                for i in range(k):
                    ###
                    #提前定义好需要的wire或者reg
                    ###l
                    para_list=[]
                    para_list+=[("wire","",f"rden_{i}")]
                    para_list+=[("wire","",f"wren_{i}")]
                    
                    
                    gv.write_v.write_parameter(para_list)
                for i in range(k):
                    ####
                   #目前没有假设清除
                    ####
                    ram_para_list=[]
                    ram_para_list+=[("rdaddress","rdaddress")]
                    ram_para_list+=[("wraddress","wraddress")]
                    ram_para_list+=[("clock","clock")]
                    ram_para_list+=[("data","data")]
                    ram_para_list+=[("rden",f"rden_{i}")]
                    ram_para_list+=[("wren",f"wren_{i}")]
                    ram_para_list+=[("wren","wrenq")]
                    with gv.module_inst(module_inst_name,str(i),ram_para_list):
                        pass
                for i in range(k):
                    wire_list=[]
                    wire_list+=[(f"wren_{i}",f"wr_en&&(ram_select_wr=={ram_number_reg}'d{i})")]
                    wire_list+=[(f"rden_{i}",f"rd_en&&(ram_select_rd=={ram_number_reg}'d{i})")]
                    
                    gv.write_v.write_wire_connect(wire_list)


                    

In [ ]:
ram_generate_k=ram_k_source(16)
ram_generate_k.generate_verilog()

In [ ]:
dma.generate_verilog()

In [5]:
from write_verilog import *
from math import *
from lt import *
from random import *
from mif import *
class degree_random:
###如果使用的k是16个这样小的东西，没办法，为了合理使用m9k，就需要用精度这个去映射，产生的mif文件比较简单。
###使用ramdom 的随机数 生成度degree，x,y,
###三个随机数生成模块，
#第一个描述如下，degree，先生成precesion的随机数，假定是20，然后使用二分法对比查找，需要一个ram，m9k的ram里面存放的是ram表。floor(log2[k]) degree,+prcesion,,或者这里使用两个ram，一个存放，另一个存放。这里有逻辑比较，
## 另一个模块是x，随机种子没有假定，暂不考虑
##再一个模块是y，随机种子没有假定，暂不考虑
#把x，y，degree，三个作为参数储存。x的长度，相当于3*ceil(log2(k))这样的长度的东西，加在包头。存进去。相当于三个字节。最多k可以是511个，如果是8字节。这个我后来在考虑，先给每个8个位来保存。把这些存放的一个fifo里面，度fifo，如果有必要就读，没必要就算了。
###输出就是一个fifo，一个巨大的FIFO。
    def __init__(self,k,kinds=1):
        self.k=k
        self.degree_wid=floor(log2(k))
        p=k
        while (1):
            for i in range (2,int(p**0.5+1)):
                if p%i==0:
                    p=p+1
                    break
            break
        self.p=p
        self.degree_random=71
        #####注释这里k的选取
        ###
        if (k<100):
            print("k  太小了，所以采用地址直接映射的办法。")
            self.rsd=RSD(k,knpPrecision=10)
            print("采用simple方法，生成了的文件列表")
            self.__simple_mif(kinds)
            self.__simple_degree_verilog()
            return
        

        self.rsd=RSD(k)
        print(kinds)
        self.__get_kinds_precision(kinds)
        print(self.precision)
    def __simple_degree_verilog(self):
        '''
        模块说明 生成9位 degree random
        @parameter clk
        @parameter degree_random
        @parameter valid

        '''

        file_name=f"simple_degree_{self.k}.v"
        print(f"file_name is {file_name}")
        
        k=self.k
        ram_number_reg=ceil(log2(self.k))
        
        with open(file_name,"w") as fd:
            #定义输入输出端口
            gv=verilog(fd)
            comment_list=[]
            comment_list+=[f"simple_degree generate，包含random71bit，这个位数是我随意定义的，"]
            comment_list+=[f"初始化，利用定义reg的时候自带的"]
            comment_list+=[f"包含a single 10bit rom or ram,9-bit wid"]
            gv.write_v.write_comment(comment_list)
            para_list=[]
            para_list+=[("input","wire","","clk")]
            para_list+=[("output","reg","","degree_valid")]
            para_list+=[("output","reg",f"[{8}:0]","degree_random")]
            
            module_name=f"simple_degree_{self.k}"
            para_list=[]
            para_list+=["input","","","clk"]
            para_list+=["output","reg","[8:0]","degree_random"]
            para_list+=["output","reg","","degree_valid"]
            
            

            with gv.module_announcement(module_name,para_list):
                #例化k个ram，并且给每个ram都写上
                
    
    def __simple_mif(self,kinds):
        '''
        kinds==1,patent_rsd
        kinds==2,lt_rsd
        kinds==3,lt_isd


        '''

        k_wid=ceil(log2(self.k))
        if kinds==1:
            file_name=f"patent_rsd_{self.k}"
            self.rsd.patent_rsd()
        elif kinds==2:
            file_name=f"lt_rsd_{self.k}"
            self.rsd.lt_rsd()
        elif kinds==3:
            file_name=f"lt_isd_{self.k}"
            self.rsd.lt_isd()
        else:
            raise Exception("invalid _kinds")
        ###简单的利用self.kind_density映射，当然因为每个都有9bit，所以会有空白是很正常的。
        print(f"生成的文件名称{file_name}.mif")
        print(f"每一行的数据格式 空白*{9-k_wid}+{k_wid}")
        self.mif=mif_quartus(1024,9,file_name)
        data=[]
        count=0
        for i in range(1024):
            if i>self.rsd.kind_density[count]:
                count+=1
            data.append(self.rsd.kind_weight[count])
        self.mif.generate(data)
    def __get_kinds_precision(self,kinds):
        '''
        kinds==1,patent_rsd
        kinds==2,lt_rsd
        kinds==3,lt_isd


        '''

        if kinds==1:
            self.rsd.patent_rsd()
        elif kinds==2:
            self.rsd.lt_rsd()
        elif kinds==3:
            self.rsd.lt_isd()
        else:
            raise Exception("invalid _kinds")
        kinds_need=len(self.rsd.kind_density)
        kinds_need_wid=ceil(log2(kinds_need))
        k_wid=ceil(log2(self.k))
        ram_data_wid=9*1024//(2**kinds_need_wid)
        self.precision=ram_data_wid-k_wid
        ##下面是生成的ram的提示
        print("需要生成一个ram")
        print(f"ram的大小是2**{kinds_need_wid}*{ram_data_wid}")
        if self.precision>30:
            print("self.precision 太长了，截取到30位精度，因为此时的k比较小")
            print(f"所以ram的形式是 空白*{self.precision-30}+{self.precision}+{ k_wid}")
        elif self.precision>10:
            print("self.precision 不需要截取")
            print(f"所以ram的形式是 {self.precision}+{ k_wid}")
        else :
            print("self.precision太短了，应该是可能取值的kinds太多了")
            print("建议用两个ram，来存放。具体请当都实现，暂时不考虑")
        
        
    
        
            

    def generate_verilog(self,):
        ##step1 选择一个lt分布，生成lt分布

        self.__generate_degree_mif()###产生mif文件
        self.__generate_degree_verilog()###例化这个rom，然后查找相应的可以使用的ram
        self.__generate_x_verilog()##产生x这个随机数0,prime(k)-1。
        self.__generate_y_verilog()## 产生y这个随机数1,prime(k)-1
        ## 写进fifo,如果能写进去，就写进去，不能就扔掉。
        ##速率主要是由degree产生来影响的。
        #生成的mif文件主要要使用9*1024，18*512，
        #所以要先根据k也就是kinds，来决定，精确度这个参数，然后再来决定。
    def __generate_degree_mif(self,):
        pass
        



In [6]:
degr=degree_random(16,2)


k  太小了，所以采用地址直接映射的办法。
每一行的数据格式 空白*5+4


In [37]:
from write_verilog import *
from math import *
class ram_n_k:
    def __init__(self,n,k,ram_name="source_ram"):
        '''
        定义k是定义的source ram的数量，n代表代表的是n个ram并排使用来使bit_wid加宽，
        因为在quartus 里面单口ram的最大位宽对应一个m9k是32位
        如果使用32位的话，
        这里的trick是address的大小是根据n*x=8 x=8/n x代表每个的地址线，
        '''
        if k//n!=k/n:
            raise Exception("n and k is not match")
        self.n=n
        self.k=k
        self.module_name="source_ram_"+f"{self.n}_{self.k}"
        self.module_name_save="source_save_"+f"{self.n}_{self.k}"
        self.name=self.module_name+".v"
        self.name_save=self.module_name_save+".v"
        self.ram_wide=32
        self.address=2
        self.ram_name=ram_name
        self.__generate_verilog_source_ram_n_k()
        self.__generate_verilog_source_save_n_k()
    def __generate_verilog_source_save_n_k(self,):
        '''
        生成verilog的接口
        实现数据流和帧之间的对接
        对于control 模块的接口
        input sys_clk
        input [7:0] data
        input data_valid
        input rst_n @这个rst对应下一次循环，让数据流重新开始

        对于ram-n-k的接口
        利用前面生成的ram_n_k的wr接口
        output [m-1:0] source_ram_wren
        output [address_wide_per_ram*n-1:0] source_ram_address
        output [n*m*data_wide_per_ram-1：0] data
        '''
        name=self.name_save
        n=self.n
        k=self.k
        m=k//n
        ram_wide=self.ram_wide
        address=self.address
        address_wide_per_ram=n*address
        data_wide_per_ram=n*self.ram_wide
        with open(name,"w") as fd:
            gv=verilog(fd)
            para_list=[]
            para_list+=[("input","wire","","sys_clk")]
            para_list+=[("input","wire","[7:0]","data")]

            para_list+=[("input","wire","","data_valid")]
            para_list+=[("input","wire","","rst_n")]
            para_list+=[("output","wire",f"[{m-1}:0]","source_ram_wren")]
            para_list+=[("output","wire",f"[{address_wide_per_ram*n-1}:0]","source_ram_address")]
            para_list+=[("output","wire",f"[{n*data_wide_per_ram-1}:0]","source_ram_data")]
            module_name_save=self.module_name_save
            with gv.module_announcement(module_name_save,para_list):
                comment_list=[]
                comment_list+=["this is the modul for source _ram_save"]
                comment_list+=[f"the data_wide is {32*n}"]
                comment_list+=[f"total {k}  raw data packets packed into {k//n} ram "]
                comment_list+=[f"input data is 8 bit wide"]
                gv.write_v.write_comment(comment_list)
                '''
                8个一组8个一组，到店就送进去，到了位宽就送进去            
                '''
                para_list=[]
                para_list+=[("reg",f"[{ceil(log2(n))-1}:0]","frame")]####代表帧的序号从0-n-1循环
                para_list+=[("reg",f"[{ceil(log2(m))-1}:0]","ram_number")]####代表从0-m-1循环
                para_list+=[("reg",f"[{address_wide_per_ram-1}:0]","address_reg")]##公用的address就是不停的家
                para_list+=[("reg",f"[{ceil(log2(5*188+4))-1}:0]","byte_counter")]
                
                para_list+=[("reg","","en")]
                para_list+=[("reg",f"[{data_wide_per_ram-1}:0]","data_reg")]##公用的data
                para_list+=[("reg","","frame_finished")]
                
                gv.write_v.write_parameter(para_list)

               
                ####最低的构成128bit的信号，也就是en信号产生
                with gv.always_begin("posedge sys_clk"):
                    with gv.if_begin("!rst_n"):
                        connect_list=[]
                        connect_list+=[("byte_counter",f"{ceil(log2(5*188+4))}'d0")]
                        connect_list+=[("en","1'b0")]
                        connect_list+=[("frame_finished","1'b0")]
                        gv.write_v.write_noblock(connect_list)
                    with gv.else_if_begin("data_valid"):
                        connect_list=[]
                        connect_list+=[("data_reg[7:0]","data")]
                        connect_list+=[(f"data_reg[{data_wide_per_ram-1}:8]",f"data_reg[{data_wide_per_ram-9}:0]")]
                        gv.write_v.write_noblock(connect_list)
                        with gv.if_begin(f"byte_counter=={ceil(log2(5*188+4))}'d{(5*188+4)-1}"):
                            ###frame wancheng 
                            connect_list=[]
                            connect_list+=[("frame_finished","1'b1")]
                            connect_list+=[("en","1'b1")]
                            connect_list+=[("byte_counter",f"{ceil(log2(5*188+4))}'d0")]
                            gv.write_v.write_noblock(connect_list)
                        with gv.else_if_begin(f"byte_counter[{ceil(log2(data_wide_per_ram//8-1)-1)}:0]=={ceil(log2(data_wide_per_ram//8-1))}'d{data_wide_per_ram//8-1}"):
                            connect_list=[]
                            connect_list+=[("frame_finished","1'b0")]
                            connect_list+=[("en","1'b1")]
                            connect_list+=[("byte_counter",f"byte_counter+1'b1")]
                            gv.write_v.write_noblock(connect_list)
                            
                        with gv.else_begin():
                            connect_list=[]
                            connect_list+=[("frame_finished","1'b0")]
                            connect_list+=[("en","1'b0")]
                            connect_list+=[("byte_counter",f"byte_counter+1'b1")]
                            gv.write_v.write_noblock(connect_list)
                    with gv.else_begin():
                        connect_list=[]
                        connect_list+=[("frame_finished","1'b0")]
                        connect_list+=[("en","1'b0")]
                        
                        gv.write_v.write_noblock(connect_list)

                ###描述address_reg
                with gv.always_begin("posedge sys_clk"):
                    with gv.if_begin("!rst_n"):
                        connect_list=[]
                        connect_list+=[("address_reg",f"{address_wide_per_ram}'d0")]
                        
                        gv.write_v.write_noblock(connect_list)
                    with gv.else_if_begin("en"):
                        with gv.if_begin("frame_finished"):
                            connect_list=[]
                            connect_list+=[(f"address_reg[{address_wide_per_ram-1}:{address_wide_per_ram-ceil(log2(n))}]",f"address_reg[{address_wide_per_ram-1}:{address_wide_per_ram-ceil(log2(n))}]+1'd1")]

                            connect_list+=[(f"address_reg[{address_wide_per_ram-1-ceil(log2(n))}:0]",f"{address_wide_per_ram-ceil(log2(n))}'d0")]
                            gv.write_v.write_noblock(connect_list)
                        with gv.else_begin():
                            connect_list=[]
                            connect_list+=[("address_reg",f"address_reg+1'b1")]
                            gv.write_v.write_noblock(connect_list)

                ##描述ram的number的
                with gv.always_begin("posedge sys_clk"):
                    with gv.if_begin("!rst_n"):
                        connect_list=[]
                        connect_list+=[("ram_number",f"{ceil(log2(m))}'d0")]
                        gv.write_v.write_noblock(connect_list)
                    with gv.else_if_begin(f"(frame_finished==1'b1)&&(address_reg[{address_wide_per_ram-1}:{address_wide_per_ram-ceil(log2(n))}]=={ceil(log2(n))}'d{n-1})"):
                        connect_list=[]
                        connect_list+=[("ram_number","ram_number+1'b1")]
                        gv.write_v.write_noblock(connect_list)
                    
                ##开始连线
                connect_list=[]
                ###连data

                connect_list+=[("source_ram_data","{"+','.join(["data_reg"]*m)+"}")]
                connect_list+=[("source_ram_address","{"+','.join(["address_reg"]*m)+"}")]
                for i in range(m):
                    connect_list+=[(f"source_ram_wren[{i}]",f"(en&&(ram_number=={ceil(log2(m))}'d{i}))")]

                gv.write_v.write_wire_connect(connect_list)










    def __generate_verilog_source_ram_n_k(self,):
        '''
        生成verilog的接口
        input [n*m*data_wide_per_ram-1：0] data
        input [m-1：0] wren
        input [m-1：0] rden
        input [address_wide_per_ram*n--1:0] address
        input clk
        output [n*m*data_wide_per_ram-1：0] q



        '''
        name=self.name
        n=self.n
        k=self.k
        m=k//n
        ram_wide=self.ram_wide
        address=self.address
        address_wide_per_ram=n*address
        data_wide_per_ram=n*self.ram_wide
        with open(name,"w") as fd:
            gv=verilog(fd)
            para_list=[]
            para_list+=[("input","wire","","ram_clk")]
            para_list+=[("input","wire",f"[{n*m*ram_wide-1}:0]","data")]
            para_list+=[("input","wire",f"[{m-1}:0]","wren")]
            para_list+=[("input","wire",f"[{m-1}:0]","rden")]
            para_list+=[("input","wire",f"[{address*m*n-1}:0]","address")]
            para_list+=[("output","wire",f"[{n*m*ram_wide-1}:0]","q")]
            module_name=self.module_name
            with gv.module_announcement(module_name,para_list):
                comment_list=[]
                comment_list+=["this is the modul for source _ram"]
                comment_list+=[f"the data_wide is {32*n}"]
                comment_list+=[f"total {k}  raw data packets packed into {k//n} ram "]
                gv.write_v.write_comment(comment_list)
                for i in range(k//n):
                    ram_para_list=[]
                    ram_para_list+=[("clock",f"ram_clk")]
                    ram_para_list+=[("data",f"data[{(i+1)*data_wide_per_ram-1}:{i*data_wide_per_ram}]")]
                    ram_para_list+=[("wren",f"wren[{i}]")]
                    ram_para_list+=[("address",f"address[{(i+1)*address_wide_per_ram-1}:{i*address_wide_per_ram}]")]
                    ram_para_list+=[("rden",f"rden[{i}]")]
                    ram_para_list+=[("q",f"q[{(i+1)*data_wide_per_ram-1}:{i*data_wide_per_ram}]")]
                    
                    with gv.module_inst(self.ram_name,f"_{i}",ram_para_list):
                        pass

        


class tx_fifo_n:
    def __init__(self,n):
        '''
        例化n个 8*1024的fifo
        '''

        self.n=n

        self.__generate_verilog_fifo_n()
    def __generate_verilog_fifo_n(self,):
        n=self.n
        file_name=f"tx_fifo_{n}.v"
        module_name=f"tx_fifo_{n}"
        comment_list=[]
        comment_list+=[f"this is {n} fifo inst "]
        comment_list+=["fifo is 8bit-wide and 1024-deep with aclr sys with wrclk"]
        wide=8
        deep=1024
        address_wide=10
        with open (file_name,"w") as fd:
            gv=verilog(fd)
            gv.write_v.write_comment(comment_list)
            para_list=[]
            para_list+=[("input","wire","","wrclk")]
            para_list+=[("input","wire",f"[{n-1}:0]","wrreq")]
            para_list+=[("input","wire",f"[{n*8-1}:0]","data")]
            para_list+=[("output","wire",f"[{n-1}:0]","wrempty")]
            para_list+=[("input","wire",f"[{n-1}:0]","rdreq")]
            para_list+=[("input","wire","","rdclk")]
            para_list+=[("output","wire",f"[{n*8-1}:0]","q")]
            para_list+=[("output","wire",f"[{n*10-1}:0]","rdusedw")]
            para_list+=[("input","wire",f"[{n-1}:0]","aclr")]
            with gv.module_announcement(module_name,para_list):
                for i in range(n):
                    name="tx_fifo"
                    name_append=f"_{i}"
                    para_list=[]
                    para_list+=[("aclr",f"aclr[{i}]")]
                    para_list+=[("data",f"data[{8*(i+1)-1}:{8*i}]")]
                    para_list+=[("rdclk","rdclk")]
                    para_list+=[("rdreq",f"rdreq[{i}]")]
                    para_list+=[("wrclk","wrclk")]
                    para_list+=[("wrreq",f"wrreq[{i}]")]
                    para_list+=[("q",f"q[{8*(i+1)-1}:{8*i}]")]
                    para_list+=[("rdusedw",f"rdusedw[{10*(i+1)-1}:{10*i}]")]
                    para_list+=[("wrempty",f"wrempty[{i}]")]
                    with gv.module_inst(name,name_append,para_list):
                        pass


        

In [33]:
ram_gg=ram_n_k(4,16)


In [38]:
fifo_n=tx_fifo_n(4)